In [26]:
import numpy as np
import scipy.signal as signal

from pytest import approx

In [4]:
%load_ext Cython

In [21]:
def python_v(cwfs, adc_to_pes, active,
             n_MAU = 100, thr_MAU = 3):
    adc_to_pes  = adc_to_pes.reshape(adc_to_pes.size, 1)
    active      = active    .reshape(    active.size, 1)
    MAU         = np.full(n_MAU, 1 / n_MAU)
    mau         = signal.lfilter(MAU, 1, cwfs, axis=1)

    # ccwf stands for calibrated corrected waveform
    ccwf        = np.where(                      active, cwfs/adc_to_pes, 0)
    ccwf_mau    = np.where((cwfs >= mau + thr) & active, cwfs/adc_to_pes, 0)

    cwf_sum     = np.sum(ccwf    , axis=0)
    cwf_sum_mau = np.sum(ccwf_mau, axis=0)
    return ccwf, ccwf_mau, cwf_sum, cwf_sum_mau

In [15]:
%%cython

cimport numpy as np
import  numpy as np
from scipy import signal


cpdef cython_v(double [:, :] CWF, double [:] adc_to_pes,
               int n_MAU = 200  , double thr_MAU = 5):
    cdef int j, k
    cdef int NPMT = CWF.shape[0]
    cdef int NWF  = CWF.shape[1]
    cdef list PMT = list(range(NPMT))

    cdef double [:] MAU = np.array(np.ones(n_MAU),
                                   dtype = np.double) * (1 / n_MAU)

    # CWF if above MAU threshold
    cdef double [:, :] pmt_thr  = np.zeros((NPMT, NWF), dtype=np.double)
    cdef double [:, :] pmt_thr_mau  = np.zeros((NPMT, NWF), dtype=np.double)
    cdef double [:]    MAU_pmt  = np.zeros(      NWF, dtype=np.double)

    for j in PMT:
        # MAU for each of the PMTs, following the waveform
        MAU_pmt = signal.lfilter(MAU, 1, CWF[j,:])

        for k in range(NWF):
            pmt_thr[j,k] = CWF[j,k] * 1 / adc_to_pes[j]
            if CWF[j,k] >= MAU_pmt[k] + thr_MAU: # >= not >: found testing!
                pmt_thr_mau[j,k] = CWF[j,k] * 1 / adc_to_pes[j]

    return np.asarray(pmt_thr), np.asarray(pmt_thr_mau)

In [22]:
NPMT, NWF  =   5, 100
n_MAU, thr = 100, 3.0

random_wfs = np.random.rand(NPMT, NWF) * 10
adc_to_pes = np.random.rand(NPMT)
active     = np.ones_like  (adc_to_pes, dtype=bool)

In [23]:
py_result = python_v(random_wfs, adc_to_pes, active, n_MAU, thr)
cy_result = cython_v(random_wfs, adc_to_pes,         n_MAU, thr)

In [28]:
py_result[1] == approx(cy_result[1])

True

In [60]:
def python_v(rwfs, adc_to_pes, thr, n_MAU=100):
    unifm = np.full(n_MAU, 1/n_MAU)
    blswf = rwfs.T - np.mean(rwfs ,           axis=1)
    mau   = signal.lfilter  (unifm, 1, blswf, axis=0)
    calwf = np.where(blswf > mau + thr * adc_to_pes, blswf / adc_to_pes, 0)
    return calwf.T

In [56]:
%%cython

cimport numpy as np
import  numpy as np
from scipy import signal



cpdef cython_v(np.ndarray[np.int16_t, ndim=2] SIPM,
                   double [:] adc_to_pes, thr,
                   int n_MAU=100):
    cdef int j, k
    cdef double [:, :] SiWF = SIPM.astype(np.double)
    cdef int NSiPM = SiWF.shape[0]
    cdef int NSiWF = SiWF.shape[1]
    cdef double [:] MAU = np.array(np.ones(n_MAU),
                                   dtype = np.double) * (1 / n_MAU)

    cdef double [:, :] siwf = np.zeros((NSiPM, NSiWF), dtype=np.double)
    cdef double [:]    MAU_ = np.zeros(        NSiWF , dtype=np.double)
    cdef double [:]    thrs = np.full ( NSiPM, thr)
    cdef double pmean

    # loop over all SiPMs. Skip any SiPM with adc_to_pes constant = 0
    # since this means SiPM is dead
    for j in range(NSiPM):
        if adc_to_pes[j] == 0:
            #print('adc_to_pes[{}] = 0, setting sipm waveform to zero'.format(j))
            continue

        # compute and subtract the baseline
        pmean = 0
        for k in range(NSiWF):
            pmean += SiWF[j,k]
        pmean /= NSiWF

        for k in range(NSiWF):
            SiWF[j,k] = SiWF[j,k] - pmean

        # MAU for each of the SiPMs, following the ZS waveform
        MAU_ = signal.lfilter(MAU, 1, SiWF[j,:])

        # threshold using the MAU
        for k in range(NSiWF):
            if SiWF[j,k]  > MAU_[k] + thrs[j] * adc_to_pes[j]:
                siwf[j,k] = SiWF[j,k] / adc_to_pes[j]

    return np.asarray(siwf)

In [57]:
random_wfs = np.random.randint(0, 10, size=(NPMT, NWF)).astype(np.int16)

In [61]:
py_result = python_v(random_wfs, adc_to_pes, thr, n_MAU)
cy_result = cython_v(random_wfs, adc_to_pes, thr, n_MAU)

In [62]:
py_result == approx(cy_result)

True